<div style='background-image: url("header.png") ; padding: 0px ; background-size: cover ; border-radius: 5px ; height: 250px'>
    <div style="float: right ; margin: 50px ; padding: 20px ; background: rgba(255 , 255 , 255 , 0.7) ; width: 50% ; height: 150px">
        <div style="position: relative ; top: 50% ; transform: translatey(-50%)">
            <div style="font-size: xx-large ; font-weight: 900 ; color: rgba(0 , 0 , 0 , 0.8) ; line-height: 100%">Tutorial by Mondaic</div>
            <div style="font-size: large ; padding-top: 20px ; color: rgba(0 , 0 , 0 , 0.5)">For Salvus version 0.11.25</div>
        </div>
    </div>
</div>

In [ ]:
%matplotlib inline

In [ ]:
# This notebook will use this variable to determine which
# remote site to run on.
import os

SALVUS_FLOW_SITE_NAME = os.environ.get("SITE_NAME", "local")

# Waveform Inversion

(Full)-Waveform Inversion (FWI) can reveal the properties of complex media that are otherwise not accessible to direct observation. This is based on measurements of mechanical waves - excited by either active or passive sources - that propagate through an object of interest and for which we record time series at certain receiver locations.
Those data contain valuable information about the object's interior that we can use to create quantitative images of its material properties.

In this tutorial, we consider a fairly simple setup in 2D, which is inspired by typical apertures in ultrasound computed tomography (USCT) to illuminate human tissue.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import time
import xarray as xr

import salvus.namespace as sn

## Step 1: Generate the domain and ground truth

This is a purely synthetic study, so we have to generate the "measurements" ourselves using a model with a few inclusions. We use a simple box domain of `20 x 20 cm` centered around the origin and insert an object with two spherical inclusions that mimics a breast phantom in a USCT acquisition. The model for density (`RHO`) and speed of sound (`VP`) is created from a structured grid.

In [ ]:
def get_spherical_inclusion():
    nx, ny = 200, 200
    x = np.linspace(-0.1, +0.1, nx)
    y = np.linspace(-0.1, +0.1, nx)
    xx, yy = np.meshgrid(x, y, indexing="ij")

    # Add 3 spherical inclusions
    vp = 1500.0 * np.ones_like(xx)
    rho = 980.0 * np.ones_like(xx)
    mask = np.sqrt(xx ** 2 + yy ** 2) < 0.05
    vp[mask] = 1480.0
    rho[mask] = 1000.0

    mask = np.sqrt(xx ** 2 + (yy - 0.025) ** 2) < 0.015
    vp[mask] = 1550.0
    rho[mask] = 1040.0

    mask = np.sqrt(xx ** 2 + (yy + 0.025) ** 2) < 0.015
    vp[mask] = 1460.0
    rho[mask] = 1010.0

    ds = xr.Dataset(
        data_vars={
            "vp": (["x", "y"], vp),
            "rho": (["x", "y"], rho),
        },
        coords={"x": x, "y": y},
    )

    return ds

Let's take a look at the model.

In [ ]:
true_model = get_spherical_inclusion()

# Plot the xarray dataset.
plt.figure(figsize=(16, 6))
plt.subplot(121)
true_model.vp.T.plot()
plt.subplot(122)
true_model.rho.T.plot()
plt.suptitle("Model with spherical inclusions")
plt.show()

This is also the starting point for setting up a project for the inversion. We can directly create the project from the model defined above. We will call this model the `true_model`.

In [ ]:
# Uncomment the following line to delete a
# potentially existing project for a fresh start
# !rm -rf project
if pathlib.Path("project").exists():
    print("Opening existing project.")
    p = sn.Project(path="project")
else:
    print("Creating new project.")
    vm = sn.model.volume.cartesian.GenericModel(
        name="true_model", data=true_model
    )
    p = sn.Project.from_volume_model(path="project", volume_model=vm)

## Step 2: Define the acquisition geometry

We assume a ring-shaped aperture with ultrasound transducers surrounding the breast phantom.
To keep the computations cheap, we will use only 5 emitters and 100 receiving transducers which are the same for every emitter. The `simple_config` has a few built-in options to create lists of sources and receivers, which we want to make use of. By defining the two rings below - one for sources and one for the receivers, we can readily create an `EventCollection`, i.e., a number of experiments characterized by the locations of the emitting and the receiving transducers.

In [ ]:
srcs = sn.simple_config.source.cartesian.collections.ScalarPoint2DRing(
    x=0, y=0, radius=0.09, count=5, f=1.0
)

for _i, s in enumerate(srcs._sources):
    all_recs = sn.simple_config.receiver.cartesian.collections.RingPoint2D(
        x=0, y=0, radius=0.09, count=100, fields=["phi"]
    )
    recs = [
        r
        for r in all_recs._receivers
        if np.sqrt(
            (s.location[0] - r.location[0]) ** 2
            + (s.location[1] - r.location[1]) ** 2
        )
        > 0.03
    ]
    p += sn.EventCollection.from_sources(
        sources=[s], receivers=recs, event_name_starting_index=_i
    )

The transducers are identical except for their location, so we use the same simulation time of `1.5 ms` (and later on also the same source time function) for each of them. Here, we also fix the time step to `400 ns`. While this is not strictly necessary and could be automatically detected, of course, it will simplify the data comparison across different simulations.

In [ ]:
wsc = sn.WaveformSimulationConfiguration(end_time_in_seconds=0.00015)
wsc.physics.wave_equation.time_step_in_seconds = 4.0e-7

Now, we put everything together to configure the simulations for the ground truth. To keep the costs low, we only consider a center frequency of `50 kHz` and a mesh accurate up to `100 kHz`.

In [ ]:
ec = sn.EventConfiguration(
    waveform_simulation_configuration=wsc,
    wavelet=sn.simple_config.stf.Ricker(center_frequency=50000.0),
)
p += sn.SimulationConfiguration(
    name="true_model_100kHz",
    #
    # Settings that influence the mesh.
    elements_per_wavelength=2,
    tensor_order=4,
    max_frequency_in_hertz=100000.0,
    #
    model_configuration=sn.ModelConfiguration(
        background_model=None, volume_models="true_model"
    ),
    # Potentially event dependent settings.
    event_configuration=ec,
)

Now it is time for the first simulation in our project. This is only to create data though. We haven't even started with the inversion yet...

In [ ]:
p.simulations.launch(
    simulation_configuration="true_model_100kHz",
    events=p.events.get_all(),
    site_name=SALVUS_FLOW_SITE_NAME,
    ranks_per_job=1,
)

In [ ]:
p.simulations.query(block=True)

In [ ]:
p.simulations.get_mesh("true_model_100kHz")

Once the simulations are finished, we can query the data and visualize it directly in the notebook. Note that you might have to execute this cell again, in case the simulations were not already finished.

In [ ]:
true_data = p.waveforms.get(
    data_name="true_model_100kHz", events=p.events.get_all()
)

Similar to the event geometry, we can now slide through the individual recordings of each event.

In [ ]:
true_data[0].plot(component="A", receiver_field="phi")

## Step 3: Create the initial model

Alright, enough of these simulations. Now, it is actually time to focus on the inversion.

**How do we start?** Well, we need a model of course.

**What do we already know about the medium?** Let's pretend we haven't seen the figures above and have no idea about the phantom. The best prior knowledge we have is that the domain is filled with water and we assume that the speed of sound is `1500 m/s` and the density is `980 kg/m^3`.

We create a homogeneous background model and set up a simulation configuration that contains the same events as the true data.

In [ ]:
bm = sn.model.background.homogeneous.IsotropicAcoustic(vp=1500.0, rho=980.0)
mc = sn.ModelConfiguration(background_model=bm)

p += sn.SimulationConfiguration(
    name="initial_model",
    #
    # Settings that influence the mesh.
    elements_per_wavelength=2,
    tensor_order=2,
    max_frequency_in_hertz=100000.0,
    #
    model_configuration=mc,
    # Potentially event dependent settings.
    event_configuration=sn.EventConfiguration(
        waveform_simulation_configuration=wsc,
        wavelet=sn.simple_config.stf.Ricker(center_frequency=50000.0),
    ),
)

## Step 4: Compute synthetics

With the model in hand, we could start iterating right away if we were brave enough. However, in all relevant cases we would want to do some quality checks first before burning many CPU hours. So let's start by just looking at the synthetic waveforms that the initial model produces.

In [ ]:
p.simulations.launch(
    simulation_configuration="initial_model",
    events=p.events.get_all(),
    site_name=SALVUS_FLOW_SITE_NAME,
    ranks_per_job=4,
)

In [ ]:
p.simulations.query(block=True)

In [ ]:
synthetic_data = p.waveforms.get(
    data_name="initial_model", events=p.events.get_all()
)

In [ ]:
synthetic_data[0].plot(component="A", receiver_field="phi")

In [ ]:
p.viz.nb.waveforms(
    ["true_model_100kHz", "initial_model"], receiver_field="phi"
)

This looks alright, and we are moving on to the next step and compute
initial misfits and gradients.

## Step 5: Manually experiment with misfits and gradients

Before triggering the iterations, most applications necessitate at least some amount of quality control, which makes it inevitable to run a few simulations manually. Let start by computing some misfits for the initial model.

**Wait!**

Do we actually need to run the simulations again? We have already computed shot gathers for the initial model, haven't we? Let's see what the function call will do.

In [ ]:
p += sn.MisfitConfiguration(
    name="L2",
    observed_data="true_model_100kHz",
    misfit_function="L2",
    receiver_field="phi",
)

In [ ]:
print(
    p.actions.inversion.compute_misfits(
        simulation_configuration="initial_model",
        misfit_configuration="L2",
        events=p.events.list(),
        store_checkpoints=False,
        site_name=SALVUS_FLOW_SITE_NAME,
        ranks_per_job=4,
    )
)

Hm. We did not include any output in the previous run, but since we are now expecting to run an adjoint simulation soon, we had to rerun the forward simulations to store some checkpoints.

If you closely study the output, you will notice that we did not repeat any simulations this time, but just queries the precomputed values.

What's next? Sensitivities!

In [ ]:
while not p.actions.inversion.compute_gradients(
    simulation_configuration="initial_model",
    misfit_configuration="L2",
    events=p.events.list(),
    site_name=SALVUS_FLOW_SITE_NAME,
    ranks_per_job=4,
):
    time.sleep(10.0)

This function returns individual gradients, which can be useful for event-dependent or batch inversions.
Again, take a close look at the verbose output. The forward runs were cached and not repeated to obtain the gradients.

Even better, in 2D you can immediately visualize them in a widget.

In [ ]:
p.viz.nb.gradients(
    simulation_configuration="initial_model",
    misfit_configuration="L2",
    events=p.events.list(),
)

We can also obtain the summed gradient over all selected events in one go.

In [ ]:
gradient = p.actions.inversion.sum_gradients(
    simulation_configuration="initial_model",
    misfit_configuration="L2",
    events=p.events.list(),
)
gradient

There is still quite a strong imprint of the source locations in the gradient. Some smoothing will help to obtain better updates.

In [ ]:
p.actions.inversion.smooth_model(
    model=gradient,
    smoothing_configuration=sn.ConstantSmoothing(
        smoothing_lengths_in_meters={
            "VP": 0.01,
            "RHO": 0.01,
        },
    ),
    ranks_per_job=4,
    site_name=SALVUS_FLOW_SITE_NAME,
)

Smooth. Feel free to play around with the smoothing length to see the effects on the gradient.
Indeed, the smoothed gradient looks much better. We will use a similar concept as a preconditioner encapsulated in the optimization method.

This toy problem fortunately does not require thorough QC and with the initial waveform fits and gradients, we feel prepared to start the inversion.

## Step 6: Initialize the inverse problem

The inverse problem is a similar entity like all the other configuration objects. We want to be able to keep track of all the individual steps, and avoid unnecessary repetitions of the same task. Fortunately, `SalvusProject` takes care of data management and book keeping.

We need to specify the prior, which is just the `SimulationConfiguration` object of the initial model we created above. Furthermore, we need to specify all possible events that we might consider during the inversion. This could be a subset of events defined in the project, and we could add more events later on.
Together with the events, we need to pass the observed data. Because we created it synthetically, this is also just a `SimulationConfiguration` object. The remaining parameters specify which parameters to invert for (`VP` and `RHO`), what misfit functional to use, preconditioner and descent method, and where to run the simulations.

In [ ]:
p += sn.InverseProblemConfiguration(
    name="my_inversion",
    prior_model="initial_model",
    events=[e.event_name for e in p.events.get_all()],
    mapping=sn.Mapping(scaling="absolute", inversion_parameters=["VP", "RHO"]),
    preconditioner=sn.ConstantSmoothing({"VP": 0.01, "RHO": 0.01}),
    method=sn.TrustRegion(initial_trust_region_linf=10.0),
    misfit_configuration="L2",
    job_submission=sn.SiteConfig(
        site_name=SALVUS_FLOW_SITE_NAME, ranks_per_job=4
    ),
)

In [ ]:
# Instead of monolithic inversions and a linear flow of iteration, `SalvusOpt` uses a tree-based framework of iteration. At any point during the inversion, you can branch off, modify the settings, and run one or more streams simultaneously.
#
# Ready for our first iteration? Without specifying any additional information, all parameters will be inherited from the InverseProblemConfiguration.

In [ ]:
p.inversions.add_iteration(inverse_problem_configuration="my_inversion")

In [ ]:
#
#
# ## Step 7: Update the model
#
# Many steps within a single iteration involve expensive simulations, e.g., for computing misfits or adjoint simulations to compute gradients. In order to be able to closely monitor the progress, `SalvusOpt` steps through an iteration, and automatically dispatches simulations whenever necessary. The function `resume` will return whenever `SalvusOpt` is waiting for other tasks to finish first. Calling it several time, will step through the iteration in sequence.

In [ ]:
p.inversions.resume(
    inverse_problem_configuration="my_inversion",
)

In [ ]:
p.inversions.resume(
    inverse_problem_configuration="my_inversion",
)

In [ ]:
p.viz.nb.iteration(
    inverse_problem_configuration="my_inversion", iteration_id=0
)

In [ ]:
# If you feel confident and don't want to be bothered with every single task, you can also tell `SalvusOpt` to run an entire iteration at once. Note the parameter `timeout_in_seconds`, which will force the cell to return even if the iteration has not been completed yet, and there might still be a few simulations running in the back.
#
# Again, you can execute the cell several times or mix it with calls to the previous one until the iteration is complete.

In [ ]:
p.inversions.iterate(
    inverse_problem_configuration="my_inversion",
    timeout_in_seconds=360,
    ping_interval_in_seconds=10,
)

p.viz.nb.inversion(inverse_problem_configuration="my_inversion")

In [ ]:
# There is still quite a strong imprint of the source locations, but we start moving into the right direction.
# The high velocity inclusion in the top part also shows up in the density update.
#
# ## Step 8: Automate
#
# The first update gave us confidence in the setup. For the moment, our work here is done.
# Let's run a few more iterations, lean back and wait for the results.
#
# We just loop over the `iterate` function and perform 3 more model updates.

In [ ]:
for i in range(2):
    p.inversions.iterate(
        inverse_problem_configuration="my_inversion",
        timeout_in_seconds=360,
        ping_interval_in_seconds=10,
    )

In [ ]:
# Done!
#
# Did we converge towards the ground truth?

In [ ]:
p.viz.nb.inversion(inverse_problem_configuration="my_inversion")

In [ ]:
# Well, at are getting closer and the heterogeneities start to form. You can, of course, continue with more iterations or play around with the settings.
#
#
# That's the end of this tutorial. We ran a few iterations of full-waveform inversion in a typical aperture for ultrasound screenings. Note that although the problem size is small and we did not apply any sophisticated inversion technique, we were able to create a suitable initial model and to perform a few model update with just a few lines of code. Nothing would change really if we applied this at a larger scale. By changing the site name to a remote site with more compute capabilities, we could easily scale up the problem.
#
# ## Additional prior knowledge
#
# In a typical USCT setup, there is always enough space between the ultrasound transducers and the phantom.
# What if we include that information as prior knowledge into our problem formulation?
#
# An easy way of doing this, is to define a region of interest and restrict the reconstruction to this area.
#
# To keep it simple, we just define a sphere with a radius of `6.5 cm` as the target region.

In [ ]:
mesh = p.simulations.get_mesh(simulation_configuration="initial_model")
# define the region of interest
roi = np.zeros_like(mesh.connectivity)
mask = np.linalg.norm(mesh.points[mesh.connectivity], axis=2) < 0.065
roi[mask] = 1.0
mesh.attach_field("region_of_interest", roi)

In [ ]:
# Let's see if this helps with the iterations. To be able to compare the results, we just create a new inverse problem within the same project, initialize the region of interest, and start iterating.

In [ ]:
p += sn.InverseProblemConfiguration(
    name="my_second_inversion",
    prior_model="initial_model",
    events=[e.event_name for e in p.events.get_all()],
    mapping=sn.Mapping(
        scaling="absolute",
        inversion_parameters=["VP", "RHO"],
        region_of_interest=mesh,
    ),
    preconditioner=sn.ConstantSmoothing({"VP": 0.01, "RHO": 0.01}),
    method=sn.TrustRegion(initial_trust_region_linf=10.0),
    misfit_configuration="L2",
    job_submission=sn.SiteConfig(
        site_name=SALVUS_FLOW_SITE_NAME, ranks_per_job=4
    ),
)

In [ ]:
p.inversions.iterate(
    inverse_problem_configuration="my_second_inversion",
    timeout_in_seconds=360,
    ping_interval_in_seconds=10,
)

In [ ]:
# Let's see if the region of interest was considered when the model was updated.

In [ ]:
p.viz.nb.inversion(inverse_problem_configuration="my_second_inversion")

In [ ]:
# Indeed, outside of the pre-defined sphere, the model is still constant and has the same values as the initial model.
#
# Let's do a few more iterations and see what the reconstruction will be.

In [ ]:
for i in range(2):
    p.inversions.iterate(
        inverse_problem_configuration="my_second_inversion",
        timeout_in_seconds=360,
        ping_interval_in_seconds=10,
    )
p.viz.nb.inversion(inverse_problem_configuration="my_second_inversion")

In [ ]:
# This looks better, so the prior knowledge was indeed helpful.
#
#
# **Bonus question.** The setup above is also a prime example of an "inverse crime", except for one small detail.
# Can you identify what it is?